<a href="https://colab.research.google.com/github/Joe941/MisionTIC/blob/main/Jenks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install jenkspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.3/523.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import jenkspy

sales = {
    'account': [
        'Jones Inc', 'Alpha Co', 'Blue Inc', 'Super Star Inc', 'Wamo',
        'Next Gen', 'Giga Co', 'IniTech', 'Beta LLC'
    ],
    'Total': [1500, 2100, 50, 20, 75, 1100, 950, 1300, 1400]
}

df = pd.DataFrame(sales)
df.sort_values(by='Total')

df['quantile'] = pd.qcut(df['Total'], q=2, labels=['bucket_1', 'bucket_2'])
df.sort_values(by='Total')

df['cut_bins'] = pd.cut(df['Total'],
                        bins=2,
                        labels=['bucket_1', 'bucket_2'])
df.sort_values(by='Total')


### Para ilustrar cómo se encuentran los quiebres naturales, podemos comenzar contrastándolo con cómo se determinan los cuantiles. Por ejemplo, ¿qué sucede si tratamos de usar pd.qcut con 2 cuantiles? ¿Eso nos dará un resultado similar?###

In [22]:
df['quantile'] = pd.qcut(df['Total'], q=2, labels=['bucket_1', 'bucket_2'])

df.sort_values(by='Total')


,account,Total,quantile
3,Super Star Inc,20,bucket_1
2,Blue Inc,50,bucket_1
4,Wamo,75,bucket_1
6,Giga Co,950,bucket_1
5,Next Gen,1100,bucket_1
7,IniTech,1300,bucket_2
8,Beta LLC,1400,bucket_2
0,Jones Inc,1500,bucket_2
1,Alpha Co,2100,bucket_2


In [23]:
df['cut_bins'] = pd.cut(df['Total'],
                        bins=2,
                        labels=['bucket_1', 'bucket_2'])
df.sort_values(by='Total')


,account,Total,quantile,cut_bins
3,Super Star Inc,20,bucket_1,bucket_1
2,Blue Inc,50,bucket_1,bucket_1
4,Wamo,75,bucket_1,bucket_1
6,Giga Co,950,bucket_1,bucket_1
5,Next Gen,1100,bucket_1,bucket_2
7,IniTech,1300,bucket_2,bucket_2
8,Beta LLC,1400,bucket_2,bucket_2
0,Jones Inc,1500,bucket_2,bucket_2
1,Alpha Co,2100,bucket_2,bucket_2


In [36]:
df['quantilev2'] = pd.qcut(
    df['Total'], q=4, labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4'])

df.sort_values(by='Total')

,account,Total,quantile,cut_bins,quantilev2
3,Super Star Inc,20,bucket_1,bucket_1,bucket_1
2,Blue Inc,50,bucket_1,bucket_1,bucket_1
4,Wamo,75,bucket_1,bucket_1,bucket_1
6,Giga Co,950,bucket_1,bucket_1,bucket_2
5,Next Gen,1100,bucket_1,bucket_2,bucket_2
7,IniTech,1300,bucket_2,bucket_2,bucket_3
8,Beta LLC,1400,bucket_2,bucket_2,bucket_3
0,Jones Inc,1500,bucket_2,bucket_2,bucket_4
1,Alpha Co,2100,bucket_2,bucket_2,bucket_4


In [42]:
df['cut_jenksv3'] = pd.cut(
    df['Total'],
    bins=jenkspy.jenks_breaks(df['Total'], n_classes=4),
    labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4'],
    include_lowest=True)
df.sort_values(by='Total')

,account,Total,quantile,cut_bins,quantilev2,cut_jenksv3
3,Super Star Inc,20,bucket_1,bucket_1,bucket_1,bucket_1
2,Blue Inc,50,bucket_1,bucket_1,bucket_1,bucket_1
4,Wamo,75,bucket_1,bucket_1,bucket_1,bucket_1
6,Giga Co,950,bucket_1,bucket_1,bucket_2,bucket_2
5,Next Gen,1100,bucket_1,bucket_2,bucket_2,bucket_2
7,IniTech,1300,bucket_2,bucket_2,bucket_3,bucket_3
8,Beta LLC,1400,bucket_2,bucket_2,bucket_3,bucket_3
0,Jones Inc,1500,bucket_2,bucket_2,bucket_4,bucket_3
1,Alpha Co,2100,bucket_2,bucket_2,bucket_4,bucket_4


In [ ]:
def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

gvf

In [ ]:
gvf = 0.0
nclasses = 2
while gvf < .8:
    gvf = goodness_of_variance_fit(array, nclasses)
    nclasses += 1